## Capstone Project: Week 5

### About the project
1. The following script helps potential clients who are looking to buy suitable property in London
2. With a pre-determined budged, this automation will recommend you locations and current average price of real estate where one can make a real estate investment
3. Important amenities/ venues for each recommended location are displayed
4. Facilities like elementary schools, high schools, hospitals & grocery stores are displayed for each recommended location¶

_Import_ Important Libraries

In [ ]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library
!conda install -c anaconda xlrd --yes
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

Installing Library for Geocoder

In [1]:
pip install geocoder

     |████████████████████████████████| 102kB 6.1MB/s ta 0:00:011
     |████████████████████████████████| 829kB 16.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 16.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [3]:
import folium # map rendering library
import time
import geocoder 

### Importing the dataset and primary processing
#### Source of dataset: http://landregistry.data.gov.uk/

In [4]:
# Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-monthly-update-new-version.csv")

# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")


In [5]:
df_affordable.head()

,Street,Avg_Price
31,AGATE ROAD,2.240000e+06
99,ANHALT ROAD,2.250000e+06
140,ASHINGTON ROAD,2.375000e+06
255,BEECH DRIVE,2.215000e+06
272,BELSIZE LANE,2.366667e+06


### Function to get the coordinates

In [ ]:
#Geocoder starts here
# Defining a function to use --> get_latlng()
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Geocoder ends here

### Getting the coordinates for the streets

In [9]:
#Applying it to our dataframe:
start = time.time()

Headquarter = df_affordable['Street']    
coordinates = [get_latlng(Headquarter) for Headquarter in Headquarter.tolist()]

end = time.time()
print("Time of execution: ", end - start, "seconds")

london_data = df_affordable

# The obtained coordinates (latitude and longitude) are joined with the dataframe as shown
df_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
london_data['Latitude'] = df_coordinates['Latitude']
london_data['Longitude'] = df_coordinates['Longitude']
#df_delhi.to_csv('DelhiHeadquarterCoordinates.csv', index = False)
london_data = london_data.dropna(how ='any',axis=0)
london_data

Time of execution:  20.01204824447632 seconds


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Street,Avg_Price,Latitude,Longitude
31,AGATE ROAD,2240000.0,51.51459,-0.19888


In [2]:
CLIENT_ID = 'GLOQ4Y14EA0FSVRORBXI0WP0NHC0GMUX5SACBXSQK3DXJDAB' # your Foursquare ID
CLIENT_SECRET = 'KICYXPZM4LFJ03EQXSRXJNBBZ0NVMTDQCTZK131M5BADYJZV' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GLOQ4Y14EA0FSVRORBXI0WP0NHC0GMUX5SACBXSQK3DXJDAB
CLIENT_SECRET:KICYXPZM4LFJ03EQXSRXJNBBZ0NVMTDQCTZK131M5BADYJZV


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Read the street-wise coordinates into a dataframe

In [ ]:
df_pref_loc = pd.merge(london_data, df_affordable, on=['Street'], how='inner')

In [ ]:
df_pref_loc.head(10)

#### Let's plot recommended locations on map of London with current market prices

In [ ]:
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Manhattan using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df_pref_loc['Latitude'], df_pref_loc['Longitude'], df_pref_loc['Avg_Price'], df_pref_loc['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '*******' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

##### Let's explore the first neighborhood in our dataframe.

## Explore the preferred location for different venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each location and create a new dataframe called location_venues and display it.

In [ ]:
# type your answer here
location_venues = getNearbyVenues(names=df_pref_loc['Street'],
                                   latitudes=df_pref_loc['Latitude'],
                                   longitudes=df_pref_loc['Longitude']
                                  )

In [ ]:
location_venues

In [ ]:
location_venues.groupby('Street').count()

### Let's explore the first neighborhood in our dataframe.

In [ ]:
def getNearbyFacility(names, latitudes, longitudes, radius=5000):
    
    facility_list=[]
    
    radius = 5000
    LIMIT = 100
    categories = '4bf58dd8d48988d196941735,58daa1558bbb0b01f18ec1f7,4bf58dd8d48988d13d941735,4f4533804b9074f6e4fb0105,4bf58dd8d48988d118951735'
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        categories,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

        # make the GET request
        results = requests.get(url).json()['response']

        # return only relevant information for each nearby venue
        facility_list.append([(
            name,
            facility['name'],
            facility['categories'][0]['name'],
            facility['location']['distance'], 
            facility['location']['lat'],
            facility['location']['lng']) for facility in results['venues']])

    nearby_facility = pd.DataFrame([item for f_list in facility_list for item in f_list])
    nearby_facility.columns = ['Street Name',
                  'Facility Name',             
                  'Facility Category', 
                  'Distance', 
                  'Facility Latitude', 
                  'Facility Longitude']
    
    return(nearby_facility)

In [ ]:
location_facility = getNearbyFacility(names=df_pref_loc['Street'],
                                      latitudes=df_pref_loc['Latitude'],
                                      longitudes=df_pref_loc['Longitude']
                                  )

In [ ]:
location_facility

In [ ]:
location_facility.groupby('Street Name').count()